# NÖ Landtagswahlen seit 1998

Quellen:

- **Landtagswahlen seit 1998:** https://www.noel.gv.at/noe/Wahlen/Landtagswahlen.html
- **Verwaltungsgrenzen:** https://www.bev.gv.at/Services/Downloads/Produktbezogene-Downloads/Unentgeltliche-Produkte/Kataster-Verzeichnisse.html
- **Umcodierung der Gemeindekennziffern Niederösterreich:** https://www.data.gv.at/katalog/dataset/7f759371-bdb3-4f58-9dbd-835eb3c19efa

In [ ]:
import pandas as pd, datetime as dt, numpy as np
import matplotlib.pyplot as plt
pd.set_option('display.max_rows', 128)

In [ ]:
def parse_bmi_file(filename, year, parties):
    type_list = []
    year_list = []
    gemeinde_list = []
    date_created_list = []
    wahlberechtigt_list = []
    wahlberechtigt_male_list = []
    wahlberechtigt_female_list = []
    stimmen_list = []
    stimmen_ungueltig_list = []
    stimmen_gueltig_list = []
    party_list = []
    count_list = []
    row = 0
    type = ""
    with open(filename, mode="r") as file:
        while True:
            line = file.readline()
            if not line: break
            row += 1
            if row == 1:
                type = line[0:1]
                continue
            gemeinde = line[0:5]
            date_created = line[5:19]
            wahlberechtigt = line[19:26]
            wahlberechtigt_male = line[26:33]
            wahlberechtigt_female = line[33:40]
            stimmen = line[40:47]
            stimmen_ungueltig = line[47:54]
            stimmen_gueltig = line[54:61]
            start = 61
            for party in parties:
                count = line[start:start+7]
                type_list.append(type)
                year_list.append(year)
                gemeinde_list.append(int(gemeinde))
                date_created_list.append(pd.to_datetime(date_created, format="%Y%m%d%H%M%S"))
                wahlberechtigt_list.append(int(wahlberechtigt))
                wahlberechtigt_male_list.append(int(wahlberechtigt_male))
                wahlberechtigt_female_list.append(int(wahlberechtigt_female))
                stimmen_list.append(int(stimmen))
                stimmen_ungueltig_list.append(int(stimmen_ungueltig))
                stimmen_gueltig_list.append(int(stimmen_gueltig))
                party_list.append(party)
                count_list.append(int(count))
                start = start + 7
    return pd.DataFrame({
        "Typ": type_list, "Jahr": year_list,
        "GemeindeKennzahl": gemeinde_list, "Crated": date_created_list, "Wahlberechtigte": wahlberechtigt_list,
        "WahlberechtigteMale": wahlberechtigt_male_list, "WahlberechtigtFemale": wahlberechtigt_female_list,
        "StimmenGesamt": stimmen_list, "StimmenGesamtUngueltig": stimmen_ungueltig_list,
        "StimmenGesamtGueltig": stimmen_gueltig_list, "Partei": party_list,
        "Stimmen": count_list})


Parteien laden. Sie bestimmem die Reihenfolge in der BMI Datei.

In [ ]:
parteien = pd.read_csv("parteien.txt", sep="\t", encoding="utf_8",
                       dtype={"Typ": "string", "Jahr": int, "Pos": int, "Partei": "string",
                              "ParteiName": "string"})

Gemeindeverzeichnis laden.

In [ ]:
gemeinden = pd.read_csv("gemeinden.csv.bz2", sep=";", encoding="utf_8",
                        usecols=["GKZ", "PG", "PB", "BL", "area"],
                        dtype={"GKZ": int, "PG": "string", "PB": "string", "BL": "string", "area": float}) \
    .groupby("GKZ") \
    .aggregate({"PG": "first",  "PB": "first", "BL": "first", "area": "sum"})
gemeinden["area"] = round(gemeinden.area / 1_000_000, 3)
gemeinden_locations = pd.read_csv("gemeinden_locations.csv", sep=";", encoding="utf_8",
                                  usecols=["GKZ", "X", "Y"], dtype={"X": float, "Y": float, "GKZ": int}) \
    .set_index("GKZ")
gemeinden_locations["X"] = gemeinden_locations.X.round(6)
gemeinden_locations["Y"] = gemeinden_locations.Y.round(6)
gemeinden = gemeinden.join(gemeinden_locations)
gemeinden = gemeinden.rename({"area": "GemeindeArea", "PG": "GemeindeName", "PB": "GemeindeBezirk",
                              "BL": "GemeindeBundesland", "X": "GemeindeLongitude", "Y": "GemeindeLatitude"},axis=1)


Manche Gemeinden haben wegen der Auflösung des Politischen Bezirkes Wien-Umgebung per 31.12.2016 eine andere Kennziffer bekommen.
Siehe https://www.data.gv.at/katalog/dataset/7f759371-bdb3-4f58-9dbd-835eb3c19efa

In [ ]:
gkz_translate = pd.read_csv("noe_umcodierung_lau2.csv", sep=";", encoding="utf_8",
                            usecols=["LAU2_CODE_2016", "LAU2_CODE_2017"],
                            dtype={"LAU2_CODE_2016": int, "LAU2_CODE_2017": int})
gkz_translate_dict = gkz_translate[gkz_translate.LAU2_CODE_2016 != gkz_translate.LAU2_CODE_2017] \
    .set_index("LAU2_CODE_2016").LAU2_CODE_2017.to_dict()

Die BMI Datendateien laden

In [ ]:
ergebnisse = None
for year, values in parteien.groupby("Jahr"):
    filename = f"lw{year%100:02}.bmi"
    parteien_list = values.sort_values("Pos").Partei
    print(f"Lese Ergebnisse aus {filename}")
    df = parse_bmi_file(f"lw{year%100:02}.bmi", year, parteien_list) \
        .astype({"Typ": "string", "Partei": "string"})
    df = df[(df.GemeindeKennzahl % 100 > 0) & (df.GemeindeKennzahl % 100 < 99)]
    df["GemeindeKennzahl"] = df.GemeindeKennzahl.replace(gkz_translate_dict)
    result = df.join(gemeinden, on="GemeindeKennzahl")
    ergebnisse = result if ergebnisse is None else pd.concat([ergebnisse, result])
    if result.GemeindeName.isna().sum() > 0:
        print("Nicht gefundene Gemeindekennzahlen:")
        print(result[result.GemeindeName.isna()].GemeindeKennzahl.unique())
ergebnisse = ergebnisse.merge(parteien[["Partei", "ParteiName"]], left_on="Partei", right_on="Partei")        

In [ ]:
ergebnisse.dtypes

In [ ]:
pd.io.clipboards.to_clipboard(ergebnisse.sample(5).to_markdown(index=False), excel=False)    # pip install tabulate --upgrade
ergebnisse.to_csv("../ltw_noe.csv.bz2", compression={'method': 'bz2', 'compresslevel': 9}, sep=";", encoding="utf-8", index=False)
ergebnisse.to_csv("../ltw_noe_unicode.csv.bz2", compression={'method': 'bz2', 'compresslevel': 9}, sep=";", encoding="utf-16", index=False)
ergebnisse.to_parquet("../ltw_noe.parquet", compression="brotli")